In [46]:
import struct

# Convert hex string to a list of 32-bit words
input_string = "486949616D4D697468696E4B756D617250656E746170616C6C6966726F6D494954424F4D424159456C656374726963616C80000000000000000000000000188"
data = [int(input_string[i:i+8], 16) for i in range(0, len(input_string), 8)]

# Define right rotate function
def rotr(x, n):
    return (x >> n) | (x << (32 - n)) & 0xFFFFFFFF

# Define SHA-256 sigma functions
def sigma0(x):
    return rotr(x, 7) ^ rotr(x, 18) ^ (x >> 3)

def sigma1(x):
    return rotr(x, 17) ^ rotr(x, 19) ^ (x >> 10)

def Sigma0(x):
    return rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22)

def Sigma1(x):
    return rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25)

def Ch(x, y, z):
    return (x & y) ^ (~x & z)

def Maj(x, y, z):
    return (x & y) ^ (x & z) ^ (y & z)

# SHA-256 constants
K = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]

# Initial hash values
H = [
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
]

# Expand the message schedule array
def expand(data):
    w = data[:]
    for i in range(16, 64):
        wj_15 = w[i - 15]
        wj_2 = w[i - 2]
        s0 = sigma0(wj_15)
        s1 = sigma1(wj_2)
        wj_7 = w[i - 7]
        wj_16 = w[i - 16]
        new_word = (wj_16 + s0 + wj_7 + s1) & 0xFFFFFFFF
        w.append(new_word)
        print(f"W[{i:2d}] = {new_word:08x} (W[j-16]={wj_16:08x}, sigma0(W[j-15])={s0:08x}, W[j-7]={wj_7:08x}, sigma1(W[j-2])={s1:08x})")
    return w

# Compression function
def compress(data):
    w = expand(data)
    a, b, c, d, e, f, g, h = H

    for i in range(64):
        T1 = (h + Sigma1(e) + Ch(e, f, g) + K[i] + w[i]) & 0xFFFFFFFF
        T2 = (Sigma0(a) + Maj(a, b, c)) & 0xFFFFFFFF
        h = g
        g = f
        f = e
        e = (d + T1) & 0xFFFFFFFF
        d = c
        c = b
        b = a
        a = (T1 + T2) & 0xFFFFFFFF

        print(f"t={i:2d} a={a:08x} b={b:08x} c={c:08x} d={d:08x} e={e:08x} f={f:08x} g={g:08x} h={h:08x}")

    # Combine with the initial hash values
    H[0] = (H[0] + a) & 0xFFFFFFFF
    H[1] = (H[1] + b) & 0xFFFFFFFF
    H[2] = (H[2] + c) & 0xFFFFFFFF
    H[3] = (H[3] + d) & 0xFFFFFFFF
    H[4] = (H[4] + e) & 0xFFFFFFFF
    H[5] = (H[5] + f) & 0xFFFFFFFF
    H[6] = (H[6] + g) & 0xFFFFFFFF
    H[7] = (H[7] + h) & 0xFFFFFFFF

    return H

# Perform the compression
compressed_hash = compress(data)

# Print the initial 16 words
for i in range(16):
    print(f"W[{i:2d}] = {data[i]:08x}")

# Print the final hash value
print("Final hash values:")
for i, h in enumerate(compressed_hash):
    print(f"H[{i}] = {h:08x}")

# Concatenate the final hash values to get the signature
final_signature = ''.join(f'{h:08x}' for h in compressed_hash)
print(f"Final SHA-256 Signature: {final_signature}")



W[16] = 49d8cf55 (W[j-16]=48694961, sigma0(W[j-15])=bf2e2caf, W[j-7]=42415945, sigma1(W[j-2])=00000000)
W[17] = 9af6f1f7 (W[j-16]=6d4d6974, sigma0(W[j-15])=c04f250f, W[j-7]=6c656374, sigma1(W[j-2])=00f50000)
W[18] = 0b413947 (W[j-16]=68696e4b, sigma0(W[j-15])=b21bebb7, W[j-7]=72696361, sigma1(W[j-2])=7e527be4)
W[19] = 42025816 (W[j-16]=756d6172, sigma0(W[j-15])=b931730b, W[j-7]=6c800000, sigma1(W[j-2])=a6e38399)
W[20] = d8a717ad (W[j-16]=50656e74, sigma0(W[j-15])=ccb7f4b3, W[j-7]=00000000, sigma1(W[j-2])=bb89b486)
W[21] = 7953305b (W[j-16]=6170616c, sigma0(W[j-15])=b0c96518, W[j-7]=00000000, sigma1(W[j-2])=671969d7)
W[22] = a2e00edc (W[j-16]=6c696672, sigma0(W[j-15])=cd612860, W[j-7]=00000188, sigma1(W[j-2])=69157e82)
W[23] = bb723854 (W[j-16]=6f6d4949, sigma0(W[j-15])=03f39867, W[j-7]=49d8cf55, sigma1(W[j-2])=fe38874f)
W[24] = ca74b77d (W[j-16]=54424f4d, sigma0(W[j-15])=d49df90a, W[j-7]=9af6f1f7, sigma1(W[j-2])=069d7d2f)
W[25] = 661a2696 (W[j-16]=42415945, sigma0(W[j-15])=bd897db1, W[

In [41]:
from hashlib import sha256

input_ = input("HiIamMithinKumarPentapallifromIITBOMBAYElectrical")
print(sha256(input_.encode('utf-8')).hexdigest())
print(input_.encode('utf-8'))

HiIamMithinKumarPentapallifromIITBOMBAYElectricalHiIamMithinKumarPentapallifromIITBOMBAYElectrical
f19e71a01190ed06b2638d6240eb7f2085a8b210ebe660e469939c918fd26591
b'HiIamMithinKumarPentapallifromIITBOMBAYElectrical'
